# arvancloud: clear

https://docs.arvancloud.ir/fa/developer-tools/sdk/object-storage/upload-object/

In [1]:
import boto3
from functools import reduce
from tqdm import tqdm

from bluer_options import string

from bluer_objects.host import signature
from bluer_objects import env
from bluer_objects.storage import S3Interface
from bluer_objects.logger import logger

logger.info(f"{'.'.join(signature())},\nbuilt on {string.pretty_date()}")

🌀  bluer_objects-6.167.1.bluer_ai-12.168.1.bluer_objects-6.167.1.bluer_options-5.137.1.torch-2.2.2.Python 3.12.9.Darwin 23.6.0..Jupyter-Notebook,
built on 02 June 2025, 22:41:05


In [2]:
storage = S3Interface()

In [3]:
try:
    s3 = boto3.client(
        "s3",
        endpoint_url=env.S3_STORAGE_ENDPOINT_URL,
        aws_access_key_id=env.S3_STORAGE_AWS_ACCESS_KEY_ID,
        aws_secret_access_key=env.S3_STORAGE_AWS_SECRET_ACCESS_KEY,
    )

    prefix = f"test"

    paginator = s3.get_paginator("list_objects_v2")
    pages = paginator.paginate(
        Bucket=env.S3_STORAGE_BUCKET,
        Prefix=prefix,
    )
except Exception as e:
    logger.error(e)
    assert False

list_of_objects = sorted(
    list(set(reduce(
        lambda x, y: x + y,
        [
            [
                obj["Key"].split("/",1)[0]
                for obj in page.get("Contents", [])
            ]
            for page in pages
        ],
        [],
    )
)))

logger.info(f"{len(list_of_objects)} object(s) to delete.")

🌀  850 object(s) to delete.


---

In [4]:
for object_name in tqdm(list_of_objects):
    assert storage.delete(
        object_name=object_name,
        do_dryrun=False,
    )
    break

  0%|          | 0/850 [00:00<?, ?it/s]


---

In [5]:
# END